In [10]:
import glob
import pandas as pd

In [42]:
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.datasets import make_blobs

In [28]:
data_f = glob.glob('C:/Users/joalam/Documents/Workspace/Lending Club/loan.csv')[0]

In [29]:
df = pd.read_csv(data_f, nrows=5)

In [30]:
print('\n'.join(x for x in df.columns))

id
member_id
loan_amnt
funded_amnt
funded_amnt_inv
term
int_rate
installment
grade
sub_grade
emp_title
emp_length
home_ownership
annual_inc
verification_status
issue_d
loan_status
pymnt_plan
url
desc
purpose
title
zip_code
addr_state
dti
delinq_2yrs
earliest_cr_line
inq_last_6mths
mths_since_last_delinq
mths_since_last_record
open_acc
pub_rec
revol_bal
revol_util
total_acc
initial_list_status
out_prncp
out_prncp_inv
total_pymnt
total_pymnt_inv
total_rec_prncp
total_rec_int
total_rec_late_fee
recoveries
collection_recovery_fee
last_pymnt_d
last_pymnt_amnt
next_pymnt_d
last_credit_pull_d
collections_12_mths_ex_med
mths_since_last_major_derog
policy_code
application_type
annual_inc_joint
dti_joint
verification_status_joint
acc_now_delinq
tot_coll_amt
tot_cur_bal
open_acc_6m
open_act_il
open_il_12m
open_il_24m
mths_since_rcnt_il
total_bal_il
il_util
open_rv_12m
open_rv_24m
max_bal_bc
all_util
total_rev_hi_lim
inq_fi
total_cu_tl
inq_last_12m
acc_open_past_24mths
avg_cur_bal
bc_open_to_buy
b

In [31]:
df[['loan_amnt', 'int_rate', 'grade', 'sub_grade', 'open_acc']]

,loan_amnt,int_rate,grade,sub_grade,open_acc
0,2500,13.56,C,C1,9
1,30000,18.94,D,D2,13
2,5000,17.97,D,D1,8
3,4000,18.94,D,D2,10
4,30000,16.14,C,C4,12


In [33]:
def get_columns_from_csv(filename, columns, chunksize=1000):
    '''
    columns: list of fields
    filename: full file path
    '''
    dfs = []
    for chunk in pd.read_csv(filename, chunksize=chunksize):
        dfs.append(chunk[columns])
    return pd.concat(dfs)

In [45]:
def fillna_with_mean(df, fields):
    for f in fields:
        df[f].fillna(df[f].mean(), inplace=True)
    return df

In [34]:
df_data = get_columns_from_csv(data_f,['loan_amnt', 'int_rate', 'grade', 'sub_grade', 'open_acc'], 3000 )

In [46]:
df_data = fillna_with_mean(df_data, ['loan_amnt', 'open_acc'])

In [47]:
x = df_data[['loan_amnt', 'open_acc']]

In [48]:
# The following bandwidth can be automatically detected using
bandwidth = estimate_bandwidth(x, quantile=0.2, n_samples=500)

In [ ]:
ms = MeanShift(bin_seeding=True)
ms.fit(x)
labels = ms.labels_
cluster_centers = ms.cluster_centers_

In [ ]:
labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)

print("number of estimated clusters : %d" % n_clusters_)

# #############################################################################
# Plot result
import matplotlib.pyplot as plt
from itertools import cycle

plt.figure(1)
plt.clf()

colors = cycle('bgrcmykbgrcmykbgrcmykbgrcmyk')
for k, col in zip(range(n_clusters_), colors):
    my_members = labels == k
    cluster_center = cluster_centers[k]
    plt.plot(X[my_members, 0], x[my_members, 1], col + '.')
    plt.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,
             markeredgecolor='k', markersize=14)
plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()